## ASOS data



In [42]:
import requests
from urllib import parse
import requests
from bs4 import BeautifulSoup
import pandas as pd
import datetime as d
import numpy as np
from tqdm import tqdm

In [2]:
url = 'http://apis.data.go.kr/1360000/AsosHourlyInfoService/getWthrDataList'
key = 'SqhNwAXk7Yr%2FbTn5Fi293%2BkNasMB8X00NPpYmWg3ruCPvqgUOR6C7IWc13dkJ2N0b9DAgyLgJEcdLC%2F3xybDLw%3D%3D'

In [3]:
params = f'?{parse.quote_plus("ServiceKey")}={key}&' + parse.urlencode({
    parse.quote_plus('pageNo') : '1', 
    parse.quote_plus('numOfRows') : '10', 
    parse.quote_plus('dataType') : 'XML', 
    parse.quote_plus('dataCd') : 'ASOS', 
    parse.quote_plus('dateCd') : 'HR', 
    parse.quote_plus('startDt') : '20211225', 
    parse.quote_plus('startHh') : '00', 
    parse.quote_plus('endDt') : '20211226', 
    parse.quote_plus('endHh') : '01', 
    parse.quote_plus('stnIds') : '104' 
    })

In [4]:
response = requests.get(url + params).text
#response = requests.get(url + params)
#.text.encode('utf-8')
print(url + params)

http://apis.data.go.kr/1360000/AsosHourlyInfoService/getWthrDataList?ServiceKey=SqhNwAXk7Yr%2FbTn5Fi293%2BkNasMB8X00NPpYmWg3ruCPvqgUOR6C7IWc13dkJ2N0b9DAgyLgJEcdLC%2F3xybDLw%3D%3D&pageNo=1&numOfRows=10&dataType=XML&dataCd=ASOS&dateCd=HR&startDt=20211225&startHh=00&endDt=20211226&endHh=01&stnIds=104


In [26]:
soup = BeautifulSoup(response, 'html.parser')
tm = soup.select('body > items > item > tm')         #현상시간
dmstMtphNo = soup.select('body > items > item > dmstmtphno') #현상번호
stnnm = soup.select('body > items > item > stnnm')    #지점

In [17]:
item = soup.select('body > items > item')
item

[<item><tm>2021-12-25 00:00</tm><rnum>1</rnum><stnid>104</stnid><stnnm>북강릉</stnnm><ta>0.4</ta><taqcflg></taqcflg><rn>11.6</rn><rnqcflg></rnqcflg><ws>1.5</ws><wsqcflg></wsqcflg><wd>320</wd><wdqcflg></wdqcflg><hm>99</hm><hmqcflg></hmqcflg><pv>6.2</pv><td>0.2</td><pa>1011.2</pa><paqcflg></paqcflg><ps>1020.7</ps><psqcflg></psqcflg><ss></ss><ssqcflg>9</ssqcflg><icsr></icsr><dsnw>12.9</dsnw><hr3fhsc>11.9</hr3fhsc><dc10tca>10</dc10tca><dc10lmcsca>10</dc10lmcsca><clfmabbrcd>St</clfmabbrcd><lcsch>1</lcsch><vs>120</vs><gndsttcd></gndsttcd><dmstmtphno>05</dmstmtphno><ts>0.4</ts><tsqcflg></tsqcflg><m005te>1.6</m005te><m01te>2.6</m01te><m02te>4.0</m02te><m03te>4.6</m03te></item>,
 <item><tm>2021-12-25 01:00</tm><rnum>2</rnum><stnid>104</stnid><stnnm>북강릉</stnnm><ta>0.2</ta><taqcflg></taqcflg><rn></rn><rnqcflg>9</rnqcflg><ws></ws><wsqcflg>9</wsqcflg><wd></wd><wdqcflg>9</wdqcflg><hm>98</hm><hmqcflg></hmqcflg><pv>6.1</pv><td>0.0</td><pa>1011.0</pa><paqcflg></paqcflg><ps>1020.5</ps><psqcflg></psqcflg><s

In [25]:
item = soup.select('body > items > item > dmstmtphno')
item

[<dmstmtphno>05</dmstmtphno>,
 <dmstmtphno>05</dmstmtphno>,
 <dmstmtphno>05</dmstmtphno>,
 <dmstmtphno>05</dmstmtphno>,
 <dmstmtphno>05</dmstmtphno>,
 <dmstmtphno>05</dmstmtphno>,
 <dmstmtphno>05</dmstmtphno>,
 <dmstmtphno>05</dmstmtphno>,
 <dmstmtphno>1905</dmstmtphno>,
 <dmstmtphno>19</dmstmtphno>]

In [28]:
rows = []
for idx in range(len(tm)):
    rows.append({"tm": tm[idx], 
                 "dmstMtphNo": dmstMtphNo[idx], 
                 "stnnm": stnnm[idx]
                 })

In [30]:
columns = ["tm", "dmstMtphNo", "stnnm"]
catalog_cd_df = pd.DataFrame(rows, columns = columns)
catalog_cd_df.head(10)

,tm,dmstMtphNo,stnnm
0,[2021-12-25 00:00],[05],[북강릉]
1,[2021-12-25 01:00],[05],[북강릉]
2,[2021-12-25 02:00],[05],[북강릉]
3,[2021-12-25 03:00],[05],[북강릉]
4,[2021-12-25 04:00],[05],[북강릉]
5,[2021-12-25 05:00],[05],[북강릉]
6,[2021-12-25 06:00],[05],[북강릉]
7,[2021-12-25 07:00],[05],[북강릉]
8,[2021-12-25 08:00],[1905],[북강릉]
9,[2021-12-25 09:00],[19],[북강릉]


## Function for downloading montly data 

In [113]:
class makingParam:
  def set_my_params(self, date, place):
    key = 'SqhNwAXk7Yr%2FbTn5Fi293%2BkNasMB8X00NPpYmWg3ruCPvqgUOR6C7IWc13dkJ2N0b9DAgyLgJEcdLC%2F3xybDLw%3D%3D'
    params = f'?{parse.quote_plus("ServiceKey")}={key}&' + parse.urlencode({
        parse.quote_plus('pageNo') : '1', 
        parse.quote_plus('numOfRows') : '24', 
        parse.quote_plus('dataType') : 'XML', 
        parse.quote_plus('dataCd') : 'ASOS', 
        parse.quote_plus('dateCd') : 'HR', 
        parse.quote_plus('startDt') : str(date), 
        parse.quote_plus('startHh') : '00', 
        parse.quote_plus('endDt') : str(int(date)+1), 
        parse.quote_plus('endHh') : '01', 
        parse.quote_plus('stnIds') : str(place) 
    })
    return params

def download_file(year, month):  
    # initialize the date
    url = 'http://apis.data.go.kr/1360000/AsosHourlyInfoService/getWthrDataList'
    param = makingParam()
    if month == 12:
        day_count = 31
    else:
        day_count = (d.date(year, month+1, 1) - d.date(year, month, 1)).days
    interval = d.timedelta(minutes=60)
    myplaces = [93, 102, 104, 108, 112, 115, 119, 131, 133, 136, 138, 143, 
             146, 152, 155, 156, 159, 165, 168, 169, 177, 184, 189]

    rows = []
    for place in tqdm(myplaces):
        for i in range(0, 1):
            day = i+1          
            start_date = d.datetime(year, month, day)
            datetime = start_date.strftime("%Y%m%d")
            my_param = param.set_my_params(datetime, place)
            response = requests.get(url + my_param).text
            soup = BeautifulSoup(response, 'html.parser')
            tm = soup.select('body > items > item > tm')                 # time
            dmstmtphno = soup.select('body > items > item > dmstmtphno') # weather index
            for idx in range(len(tm)):
                if dmstmtphno[idx].text:
                    rows.append({"time": tm[idx].text, 
                                "point": place,
                                "num": dmstmtphno[idx].text                                
                                })     
    return rows

In [114]:
my_data = download_file(2022, 1) 

100%|██████████| 23/23 [00:25<00:00,  1.11s/it]


In [115]:
my_data[:2]

[{'num': '01', 'point': 102, 'time': '2022-01-01 16:00'},
 {'num': '01', 'point': 102, 'time': '2022-01-01 17:00'}]

In [117]:
columns = ["time", "point", "num"]
df = pd.DataFrame(my_data, columns = columns)
df.head(5)

,time,point,num
0,2022-01-01 16:00,102,01
1,2022-01-01 17:00,102,01
2,2022-01-01 18:00,102,01
3,2022-01-01 19:00,102,01
4,2022-01-01 20:00,102,01


In [118]:
df.to_csv("./test.csv", header=True, encoding='utf-8')